In [ ]:
%pip install -U langchain langchain-core langchain-openai langchain-community python-dotenv
%pip install -U langchain-classic
%pip install grandalf

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableSequence, RunnableLambda, RunnableParallel
from langchain_core.tracers.context import collect_runs
from langchain_core.runnables.graph_ascii import draw_ascii 
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("API_KEY")
base_url = os.getenv("OPENAI_ENDPOINT")
model_name = "gpt-4o-mini"
temp=0.0

llm = ChatOpenAI(
    base_url=base_url,
    api_key=api_key,
    model=model_name,
    temperature=temp
)

## **Chaining invocations**

In [9]:
prompt = PromptTemplate(
    template="Tell me a joke about {topic}"
)

parser = StrOutputParser()

response = parser.invoke(
    llm.invoke(
        prompt.invoke(
            {"topic": "Python"}
        )
    )
)
response

'Why do Python programmers prefer dark mode?\n\nBecause light attracts bugs!'

## **Runnables**

Runnables can be 
- executed
    - invoke(), 
    - batch() 
    - and stream()
- inspected,
- and composed

In [10]:
runnables = [prompt, llm, parser]

#### Execute methods

In [12]:
for runnable in runnables:
    print(f"{repr(runnable).split('(')[0]}")
    print(f"\tINVOKE: {repr(runnable.invoke)}")
    #print(f"\tBATCH: {repr(runnable.batch)}")
    #print(f"\tSTREAM: {repr(runnable.stream)}\n")

PromptTemplate
	INVOKE: <bound method BasePromptTemplate.invoke of PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='Tell me a joke about {topic}')>
ChatOpenAI
	INVOKE: <bound method BaseChatModel.invoke of ChatOpenAI(profile={'max_input_tokens': 128000, 'max_output_tokens': 16384, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'pdf_inputs': True, 'pdf_tool_message': True, 'image_tool_message': True, 'tool_choice': True}, client=<openai.resources.chat.completions.completions.Completions object at 0x766be29c80d0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x766be23fa610>, root_client=<openai.OpenAI object at 0x766be29bbc50>, root_async_client=<openai.AsyncOpenAI object at 0x766be23fa250>, model_name='gpt-4o-mini

#### Inspect

In [13]:
for runnable in runnables:
    print(f"{repr(runnable).split('(')[0]}")
    print(f"\tINPUT: {repr(runnable.get_input_schema())}")
    print(f"\tOUTPUT: {repr(runnable.get_output_schema())}")
    print(f"\tCONFIG: {repr(runnable.config_schema())}\n")

PromptTemplate
	INPUT: <class 'langchain_core.utils.pydantic.PromptInput'>
	OUTPUT: <class 'langchain_core.prompts.prompt.PromptTemplateOutput'>
	CONFIG: <class 'langchain_core.utils.pydantic.PromptTemplateConfig'>

ChatOpenAI
	INPUT: <class 'langchain_openai.chat_models.base.ChatOpenAIInput'>
	OUTPUT: <class 'langchain_openai.chat_models.base.ChatOpenAIOutput'>
	CONFIG: <class 'langchain_core.utils.pydantic.ChatOpenAIConfig'>

StrOutputParser
	INPUT: <class 'langchain_core.output_parsers.string.StrOutputParserInput'>
	OUTPUT: <class 'langchain_core.output_parsers.string.StrOutputParserOutput'>
	CONFIG: <class 'langchain_core.utils.pydantic.StrOutputParserConfig'>



#### <mark>Config</mark>

In [ ]:
with collect_runs() as run_collection:
    result = llm.invoke(
        "Hello", 
        config={
            'run_name': 'demo_run', 
            'tags': ['demo', 'lcel'], 
            'metadata': {'lesson': 2}
        }
    )

run_collection.traced_runs

run_collection.traced_runs[0].dict()


#### <mark>Compose Runnables</mark>

In [27]:
chain = RunnableSequence(prompt, llm, parser)

print(type(chain))

chain.invoke({"topic": "Python"})

<class 'langchain_core.runnables.base.RunnableSequence'>


'Why do Python programmers prefer dark mode?\n\nBecause light attracts bugs!'

In [28]:
for chunk in chain.stream({"topic": "Python"}):
    print(chunk, end="", flush=True)

Why do Python programmers prefer dark mode?

Because light attracts bugs!

In [29]:
chain.batch([
    {"topic": "Python"},
    {"topic": "Data"},
    {"topic": "Machine Learning"},
])

['Why do Python programmers prefer dark mode?\n\nBecause light attracts bugs!',
 'Why did the data break up with the database?\n\nBecause it found someone more relational!',
 'Why did the neural network break up with the decision tree?\n\nBecause it found someone with more layers!']

In [ ]:
chain.get_graph().print_ascii()

#### <mark>Turn any function into a runnable</mark>

In [35]:
def my_function(x:int)->int:
    return 2*x

In [36]:
my_runnable = RunnableLambda(my_function)
my_runnable.invoke(2)

4

#### Parallel Runnables

In [37]:
parallel_chain = RunnableParallel(
    double=RunnableLambda(lambda x: x * 2),
    triple=RunnableLambda(lambda x: x * 3),
)

In [38]:
parallel_chain.invoke(3)

{'double': 6, 'triple': 9}

In [24]:
parallel_chain.get_graph().print_ascii()

+------------------------------+   
| Parallel<double,triple>Input |   
+------------------------------+   
           **        **            
         **            **          
        *                *         
  +--------+          +--------+   
  | Lambda |          | Lambda |   
  +--------+          +--------+   
           **        **            
             **    **              
               *  *                
+-------------------------------+  
| Parallel<double,triple>Output |  
+-------------------------------+  


## **LCEL**

In [25]:
prompt

PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='Tell me a joke about {topic}')

In [26]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x12025b790>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x11559cdd0>, root_client=<openai.OpenAI object at 0x117ed64d0>, root_async_client=<openai.AsyncOpenAI object at 0x120471710>, model_name='gpt-4o-mini', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'))

In [27]:
parser

StrOutputParser()

In [28]:
chain = RunnableSequence(prompt, llm, parser)
chain

PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='Tell me a joke about {topic}')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x12025b790>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x11559cdd0>, root_client=<openai.OpenAI object at 0x117ed64d0>, root_async_client=<openai.AsyncOpenAI object at 0x120471710>, model_name='gpt-4o-mini', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser()

In [39]:
prompt | llm | parser

PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='Tell me a joke about {topic}')
| ChatOpenAI(profile={'max_input_tokens': 128000, 'max_output_tokens': 16384, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'pdf_inputs': True, 'pdf_tool_message': True, 'image_tool_message': True, 'tool_choice': True}, client=<openai.resources.chat.completions.completions.Completions object at 0x766be29c80d0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x766be23fa610>, root_client=<openai.OpenAI object at 0x766be29bbc50>, root_async_client=<openai.AsyncOpenAI object at 0x766be23fa250>, model_name='gpt-4o-mini', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'), openai_api_base='https://farra-ai-foundry-servi

In [40]:
chain = prompt | llm | parser

In [41]:
chain.invoke(
    {"topic": "computer"}
)

'Why did the computer go to therapy?\n\nBecause it had too many bytes from its past!'